# Deep Learning A Z Regressão 1 valor - Automóveis Cruzada

In [2]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

In [3]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.2.2', '2.17.0', '1.5.1', '0.13.0')

In [4]:
import time
from scikeras.wrappers import KerasRegressor
from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics

In [5]:
inicio = time.time()

In [6]:
inicio

1741284827.4709597

Carregamento e processamento dos dados

In [7]:
base = pd.read_csv('autos.csv', encoding='ISO-8859-1')

In [8]:
base = base.drop('dateCrawled', axis=1)
base = base.drop('dateCreated', axis=1)
base = base.drop('nrOfPictures', axis=1)
base = base.drop('postalCode', axis=1)
base = base.drop('lastSeen', axis=1)
base = base.drop('name', axis=1)
base = base.drop('seller', axis=1)
base = base.drop('offerType', axis=1)

In [9]:
base = base[base.price > 10]
base = base.loc[base.price < 350000]

In [10]:
valores = {'vehicleType': 'limousine',
           'gearbox': 'manuell',
           'model': 'golf',
           'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value=valores)

In [11]:
X = base.iloc[:, 1:12].values
y = base.iloc[:, 0].values

In [12]:
X, y

(array([['test', 'limousine', 1993, ..., 'benzin', 'volkswagen', 'nein'],
        ['test', 'coupe', 2011, ..., 'diesel', 'audi', 'ja'],
        ['test', 'suv', 2004, ..., 'diesel', 'jeep', 'nein'],
        ...,
        ['test', 'bus', 1996, ..., 'diesel', 'volkswagen', 'nein'],
        ['test', 'kombi', 2002, ..., 'diesel', 'volkswagen', 'nein'],
        ['control', 'limousine', 2013, ..., 'benzin', 'bmw', 'nein']],
       dtype=object),
 array([  480, 18300,  9800, ...,  9200,  3400, 28990], dtype=int64))

In [13]:
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])], remainder='passthrough') # Transformação das variáveis categóricas em valores numéricos
X = onehotencoder.fit_transform(X).toarray()

In [14]:
X.shape

(359291, 316)

In [15]:
def criar_rede():
    k.clear_session()
    regressor = Sequential([
        tf.keras.layers.InputLayer(shape=(316,)),
        tf.keras.layers.Dense(units=158, activation='relu'),
        tf.keras.layers.Dense(units=158, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='linear')])
    regressor.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
    return regressor

In [16]:
regressor = KerasRegressor(model = criar_rede, epochs = 100, batch_size = 300)

In [17]:
resultados = cross_val_score(estimator = regressor, X = X, y = y,
                             cv = 5, scoring = 'neg_mean_absolute_error')


Epoch 1/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 4288.4502 - mean_absolute_error: 4288.4502
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3443.4395 - mean_absolute_error: 3443.4395
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3298.1052 - mean_absolute_error: 3298.1052
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 3078.3020 - mean_absolute_error: 3078.3020
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2947.4604 - mean_absolute_error: 2947.4604
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 2872.6265 - mean_absolute_error: 2872.6265
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 2833.2019 - mean_absolute_error: 2833.2019
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2795.0696 - mean_absolute_error: 2795.0696
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2739.3970 - mean_absolute_error: 2739.3970
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2

In [18]:
fim = time.time()

In [19]:
fim

1741286243.2691464

In [20]:
(fim - inicio) / 60 / 60 # Verificando o tempo de execução

0.39327727410528396

In [21]:
abs(resultados) # Cada valor no array representa o erro médio absoluto de cada validação cruzada

array([2430.82427947, 2312.93997348, 2485.35495923, 2242.69710733,
       2253.58315565])

In [24]:
abs(resultados.mean()) # Média dos erros médios absolutos

2345.079895033073

In [23]:
resultados.std() # Desvio padrão dos erros médios absolutos

96.86697702690131